In [8]:
from openai import OpenAI
from pinecone import Pinecone
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer
import os


In [6]:
load_dotenv()

True

In [34]:
pc = Pinecone(os.getenv("PINECONE_KEY"))

In [37]:
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv('OPEN_ROUTER_API_KEY')
)

In [10]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [15]:
def embed_input(input_text):
    return model.encode(input_text).tolist()

In [26]:
query = "Does Aneesh Know React??"
query_embedding = embed_input(query)
print(query_embedding)

[-0.006735945586115122, -0.01890672743320465, 0.032685182988643646, 0.02298353612422943, 0.018526574596762657, 0.0325140543282032, 0.14394767582416534, -0.059521354734897614, 0.013226252049207687, 0.032381948083639145, -0.018215369433164597, -0.10283300280570984, -0.0007878976175561547, 0.04103735834360123, -0.021458638831973076, 0.04795558750629425, 0.01078353263437748, -0.07229714840650558, -0.0527721606194973, -0.017991023138165474, -0.05352240800857544, 0.022103141993284225, 0.054703403264284134, -0.02578096091747284, -0.0763019248843193, 0.03476263955235481, 0.023515848442912102, -0.004502199590206146, 0.029056768864393234, -0.01990315690636635, 0.012957845814526081, 0.0795958936214447, -0.011128543876111507, 0.02769375778734684, -0.027383478358387947, 0.11591824144124985, 0.05420941114425659, 0.026133401319384575, -0.06785369664430618, 0.027747979387640953, -0.002612555166706443, -0.037024665623903275, 0.03141096979379654, -0.04272883012890816, -0.025615885853767395, 0.0481992140

In [19]:
index = pc.Index("rag-resume-data")

In [24]:
def revelant_data(query_embedding):
    return index.query(vector=query_embedding, top_k=5, include_metadata=True)

In [27]:
revelant_data(query_embedding)

{'matches': [{'id': 'doc-18',
              'metadata': {'prefix': 'skills.technical_skills[2]',
                           'text': 'React'},
              'score': 0.346437812,
              'values': []},
             {'id': 'doc-1',
              'metadata': {'prefix': 'contact.name', 'text': 'Aneesh Patne'},
              'score': 0.344072491,
              'values': []},
             {'id': 'doc-5',
              'metadata': {'prefix': 'contact.leetcode',
                           'text': 'https://leetcode.com/aneeshpatne'},
              'score': 0.343364656,
              'values': []},
             {'id': 'doc-4',
              'metadata': {'prefix': 'contact.github',
                           'text': 'https://github.com/aneeshpatne'},
              'score': 0.271438152,
              'values': []},
             {'id': 'doc-3',
              'metadata': {'prefix': 'contact.linkedin',
                           'text': 'https://www.linkedin.com/in/aneeshpatne'},
              

In [38]:
try:
    response = client.chat.completions.create(
        model="mistralai/mistral-7b-instruct:free",
        messages=[
            {"role": "system", "content": "You are a helpful assistant called RESUME.AI"},
            {"role": "user", "content": query}
        ]

    )
    print(response.choices[0].message.content.strip())
except Exception as e:
    print(e)

Error code: 401 - {'error': {'message': 'No auth credentials found', 'code': 401}}
